## 대용량 비정형 데이터 기반 Advanced RAG 구축 및 평가
#### 강의 목표
1. **Real-World Data Handling**: 나무위키(구어체/지식) FinePDFs(문어체/전문문서)가 섞인 두개 데이터를 처리합니다.
2. **Beyond Keyword Search**: 단순 키워드 매칭(BM25)과 의미 기반 검색(Vector Search)의 결정적 차이를 체감합니다.
3. **Hybrid Search**: BM25와 Vector Search를 결합하여 검색 성능을 향상시킵니다.
4. **Advanced Pipeline**: Retriever + Reranker 구조를 통해 검색 정확도를 극대화하는 기법을 익힙니다.
5. **Query Decomposition**: 복잡한 질의를 분해하여 더 정확한 검색 결과를 얻는 방법을 배웁니다.
6. **Data-Driven Evalution**: 정답지가 없는 상황에서 LLM을 이용해 *합성 데이터셋(Synthetic Testset)*을 만들고 평가를 수행합니다.

#### 사용 모델 및 데이터
- **LLM**: gpt-5-mini (High Performance & Low Cost)
- **Data**: heegyu/namuwiki (General Knowledge) + HuggingFaceFW/finepdfs-edu (Domain Documents)

In [1]:
# 1. 필수 라이브러리 설치 (Jupyter 환경 권장 방식)

# 버전을 명시하여 호환성 문제 방지

%pip install -q \
    langchain==0.3.14 \
    langchain-openai==0.2.14 \
    langchain-huggingface==0.1.2 \
    langchain-community==0.3.14 \
    ragas==0.2.10 \
    datasets \
    sentence-transformers \
    tiktoken \
    rank_bm25 \
    pandas \
    faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# API_KEY 정보 로드
load_dotenv()


True

In [ ]:
# print API key
# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)

## 1. Data Loading: '현실적인' 데이터 믹싱(Mixing)
현실에는 깔끔하게 정제된 데이터만 존재하지 않습니다. **지식 백과(Wiki)** 스타일과 **보고서(PDF)** 스타일의 텍스트가 섞여서, 난이도가 높은 통합 검색 환경을 구축합니다.

In [8]:
from datasets import load_dataset
from langchain.docstore.document import Document

print("📥 데이터 스트리밍 시작...")

raw_docs = []

# [Source A] 나무위키 (General Knowledge)
# 특징: 구어체, 다양한 주제, 메타데이터에 제목 포함
wiki_stream = load_dataset("heegyu/namuwiki", split="train", streaming=True)
for idx, data in enumerate(wiki_stream):
    # if idx >= 10: break  # 실습 속도를 위해 100개만 사용, Memory & Computation 여유가 있다면 전부 써보기
    # print(f"[{idx+1}/100] {data}")
    content = f"[출처: 나무위키] 제목: {data['title']}\n\n{data['text']}"
    raw_docs.append(Document(page_content=content, metadata={"source": "namuwiki", "title": data['title']}))

length_of_wiki = len(raw_docs)

# for i, doc in enumerate(raw_docs):
#     print(f"[{i+1}/10] {doc}")

# [Source B] FinePDFs-Edu (Domain Documents)
# 특징: 문어체, 전문용어, 서식(Header/Footer) 노이즈 존재
pdf_stream = load_dataset("HuggingFaceFW/finepdfs-edu", "kor_Hang", split="train", streaming=True)
for idx, data in enumerate(pdf_stream):
    # if idx >= 100: break
    # PDF는 제목이 명확하지 않은 경우가 많아 URL을 메타데이터로 사용
    raw_docs.append(Document(page_content=data['text'], metadata={"source": "finepdf", "url": data['url']}))

length_of_pdf = len(raw_docs) - length_of_wiki

print(f"✅ 데이터 로드 완료: 총 {len(raw_docs)}개 문서")
print(f"   - Wiki 예시(총 {length_of_wiki}개): {raw_docs[0].page_content[:80]}...")
print(f"   - PDF 예시(총 {length_of_pdf}개): {raw_docs[length_of_wiki].page_content[:80]}...")




📥 데이터 스트리밍 시작...
✅ 데이터 로드 완료: 총 986677개 문서
   - Wiki 예시(총 867024개): [출처: 나무위키] 제목: !

#redirect 느낌표
...
   - PDF 예시(총 119653개): [출처: 나무위키] 제목: 04-ALICIA

[[파일:external/vignette4.wikia.nocookie.net/Alicia_sche...


---

## 2. Advanced Chunking Strategy
PPT에서 강조한 **Chunking** 전략입니다.
- **Attention Dilution 방지**: 너무 길면 LLM이 핵심을 놓칩니다.
- **Context Overlap**: 문맥이 끊기는 것을 방지하기 위해 Chunk_overlap을 둡니다.
- **Separators**: 문단(\n\n) → 줄바꿈(\n) → 문장(.) 순서로 의미 단위를 보존하며 자릅니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PPT 실무 팁 반영: 의미 단위 보존을 위한 계층적 Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 검색에 적절한 크기 (토큰 기준 아님, 글자 수 기준)
    chunk_overlap=100,  # 앞뒤 문맥 연결을 위한 오버랩
    separators=["\n\n", "\n", ". ", " ", ""]  # 의미 단위 우선 순위
)

splitted_docs = text_splitter.split_documents(raw_docs)
print(f"✂️ 청킹 결과: {len(raw_docs)}개 문서 → {len(splitted_docs)}개 청크로 분할됨")

---

## 3. 검색 엔진 구축: BM25, Vector, Hybrid
세가지 검색 방식을 구축하고 비교합니다.

|방식|특징|장점|단점|
|----|----|----|----|
|**BM25**|키워드 매칭|정확한 용어 검색에 강함|동의어/유사어 인식 불가|
|**Vector**|의미 기반|문맥적 유사성 파악|특정 키워드 검색에 약함|
|**Hybrid**|BM25 + Vector 결합|두 장점 결합|가중치 튜닝 필요|


In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 1. BM25 Retriever 구축(키워드 기반)
bm25_retriever = BM25Retriever.from_documents(splitted_docs)
bm25_retriever.k = 5  # 후보군을 넉넉하게

# 2. Vector Store 구축 (의미 기반, FAISS 사용)
# OpenAI Embedding 사용
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.from_documents(
    documents=splitted_docs,
    embedding=openai_embedding
)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print("✅ BM25 & Vector(FAISS) 검색 엔진 준비 완료!")

#### 3.1 🆕 Hybrid Search 구현 (PPT 23-24 슬라이드)
**λ•BM25 + (1-λ)•Vector** 하이브리드 검색입니다.
```
최종 스코어 = α * BM25_score + (1 - α) * Vector_score
```
LangChain의 EnsembleRetriever를 사용하여 두 검색 결과를 RRF(Reciprocal Rank Fusion) 방식으로 결합합니다.

In [ ]:
from langchain.retrievers import EnsembleRetriever

# Hybrid Retriever: BM25(키워드) + Vector(의미) 결합
# weights: [BM25 가중치, Vector 가중치] - 합이 1이 될 필요는 없음
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.4, 0.6]  # Vector에 약간 더 가중치 (사전 정의된 질문 Coverage에 따라 조정)
)

print("✅ Hybrid Search 엔진 준비 완료!")
print("   - BM25 가중치: 0.4 (키워드 매칭)")
print("   - Vector 가중치: 0.6 (의미 기반)")

#### 3.2 검색 방식 비교 실험
동일한 질문에 대해 세가지 검색 방식의 결과를 비교해봅니다.

In [ ]:
import pandas as pd

def compare_retrievers(query: str):
    """세가지 검색 방식 결과 비교"""
    print(f"🔍 질문: '{query}'\n")
    print("=" * 80)

    results = {
        "BM25 (키워드)": bm25_retriever.invoke(query),
        "Vector (의미)": vector_retriever.invoke(query),
        "Hybrid (결합)": hybrid_retriever.invoke(queyr)
    }

    for method, docs in results.items():
        print(f"\n📌 [{method}] 상위 3개 결과:")
        print("-" * 40)
        for i, doc in enumerate(docs[:3], 1):
            source = doc.metadata.get('source', 'unknown')
            preview = doc.page_content[:100].replace('\n', ' ')
            print(f"   {i}. [{source}] {preview}...")
        if not docs:
            print("   → 결과 없음")
    
    return results

# 테스트 1: 의미론적 질문(Vector가 유리)
results1 = compare_retrievers("프린터 추천")


In [ ]:
# 테스트 2: 특정 키워드 질문 (BM25가 유리)
results2 = compare_retrievers("대한민국 헌법 제1조")

In [ ]:
# 테스트 3: 복합적인 질문 (Hybrid가 유리)
results3 = compare_retrievers("인공지능 기술의 발전과 미래 전망")